<a href="https://colab.research.google.com/github/autowiki4/Deep-Chem-Tutorials/blob/main/examples/tutorials/Advanced_Model_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  Advanced Model Training

In the tutorials so far we have followed a simple procedure for training models: load a dataset, create a model, call `fit()`, evaluate it, and call ourselves done.  That's fine for an example, but in real machine learning projects the process is usually more complicated.  In this tutorial we will look at a more realistic workflow for training a model.

## Colab

This tutorial and the rest in this sequence can be done in Google colab. If you'd like to open this notebook in colab, you can use the following link.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/deepchem/deepchem/blob/master/examples/tutorials/Advanced_Model_Training.ipynb)

## Setup

To run DeepChem within Colab, you'll need to run the following installation commands. You can of course run this tutorial locally if you prefer. In that case, don't run these cells since they will download and install DeepChem in your local machine again.

In [1]:
!pip install --pre deepchem
import deepchem
deepchem.__version__

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.
Instructions for updating:
experimental_relax_shapes is deprecated, use reduce_retracing instead
wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


'2.8.1.dev'

## Hyperparameter Optimization

Let's start by loading the HIV dataset.  It classifies over 40,000 molecules based on whether they inhibit HIV replication.

In [2]:
import deepchem as dc

tasks, datasets, transformers = dc.molnet.load_hiv(featurizer='ECFP', split='scaffold')
train_dataset, valid_dataset, test_dataset = datasets

Streaming output truncated to the last 5000 lines.
[16:51:57] DEPRECATION WARNING: please use MorganGenerator
[16:51:57] DEPRECATION WARNING: please use MorganGenerator
[16:51:57] DEPRECATION WARNING: please use MorganGenerator
[16:51:57] DEPRECATION WARNING: please use MorganGenerator
[16:51:57] DEPRECATION WARNING: please use MorganGenerator
[16:51:57] DEPRECATION WARNING: please use MorganGenerator
[16:51:57] DEPRECATION WARNING: please use MorganGenerator
[16:51:57] DEPRECATION WARNING: please use MorganGenerator
[16:51:57] DEPRECATION WARNING: please use MorganGenerator
[16:51:57] DEPRECATION WARNING: please use MorganGenerator
[16:51:57] DEPRECATION WARNING: please use MorganGenerator
[16:51:57] DEPRECATION WARNING: please use MorganGenerator
[16:51:57] DEPRECATION WARNING: please use MorganGenerator
[16:51:57] DEPRECATION WARNING: please use MorganGenerator
[16:51:57] DEPRECATION WARNING: please use MorganGenerator
[16:51:57] DEPRECATION WARNING: please use MorganGenerator
[16:5

Now let's train a model on it.  We will use a `MultitaskClassifier`, which is just a stack of dense layers.  But that still leaves a lot of options.  How many layers should there be, and how wide should each one be?  What dropout rate should we use?  What learning rate?

These are called hyperparameters.  The standard way to select them is to try lots of values, train each model on the training set, and evaluate it on the validation set.  This lets us see which ones work best.

You could do that by hand, but usually it's easier to let the computer do it for you.  DeepChem provides a selection of hyperparameter optimization algorithms, which are found in the `dc.hyper` package.  For this example we'll use `GridHyperparamOpt`, which is the most basic method.  We just give it a list of options for each hyperparameter and it exhaustively tries all combinations of them.

The lists of options are defined by a `dict` that we provide.  For each of the model's arguments, we provide a list of values to try.  In this example we consider three possible sets of hidden layers: a single layer of width 500, a single layer of width 1000, or two layers each of width 1000.  We also consider two dropout rates (20% and 50%) and two learning rates (0.001 and 0.0001).

In [3]:
params_dict = {
    'n_tasks': [len(tasks)],
    'n_features': [1024],
    'layer_sizes': [[500], [1000], [1000, 1000]],
    'dropouts': [0.2, 0.5],
    'learning_rate': [0.001, 0.0001]
}
optimizer = dc.hyper.GridHyperparamOpt(dc.models.MultitaskClassifier)
metric = dc.metrics.Metric(dc.metrics.roc_auc_score)
best_model, best_hyperparams, all_results = optimizer.hyperparam_search(
        params_dict, train_dataset, valid_dataset, metric, transformers)

`hyperparam_search()` returns three arguments: the best model it found, the hyperparameters for that model, and a full listing of the validation score for every model.  Let's take a look at the last one.

In [4]:
all_results

{'_dropouts_0.200000_layer_sizes[500]_learning_rate_0.001000_n_features_1024_n_tasks_1': 0.7791613452532993,
 '_dropouts_0.200000_layer_sizes[500]_learning_rate_0.000100_n_features_1024_n_tasks_1': 0.7652253675986413,
 '_dropouts_0.500000_layer_sizes[500]_learning_rate_0.001000_n_features_1024_n_tasks_1': 0.7661043578929958,
 '_dropouts_0.500000_layer_sizes[500]_learning_rate_0.000100_n_features_1024_n_tasks_1': 0.7627690950687724,
 '_dropouts_0.200000_layer_sizes[1000]_learning_rate_0.001000_n_features_1024_n_tasks_1': 0.7761652746768102,
 '_dropouts_0.200000_layer_sizes[1000]_learning_rate_0.000100_n_features_1024_n_tasks_1': 0.7707994523921093,
 '_dropouts_0.500000_layer_sizes[1000]_learning_rate_0.001000_n_features_1024_n_tasks_1': 0.784168067844575,
 '_dropouts_0.500000_layer_sizes[1000]_learning_rate_0.000100_n_features_1024_n_tasks_1': 0.7608181653910588,
 '_dropouts_0.200000_layer_sizes[1000, 1000]_learning_rate_0.001000_n_features_1024_n_tasks_1': 0.7246999335397583,
 '_dropou

We can see a few general patterns.  Using two layers with the larger learning rate doesn't work very well.  It seems the deeper model requires a smaller learning rate.  We also see that 20% dropout usually works better than 50%.  Once we narrow down the list of models based on these observations, all the validation scores are very close to each other, probably close enough that the remaining variation is mainly noise.  It doesn't seem to make much difference which of the remaining hyperparameter sets we use, so let's arbitrarily pick a single layer of width 1000 and learning rate of 0.0001.

## Early Stopping

There is one other important hyperparameter we haven't considered yet: how long we train the model for.  `GridHyperparamOpt` trains each for a fixed, fairly small number of epochs.  That isn't necessarily the best number.

You might expect that the longer you train, the better your model will get, but that isn't usually true.  If you train too long, the model will usually start overfitting to irrelevant details of the training set.  You can tell when this happens because the validation set score stops increasing and may even decrease, while the score on the training set continues to improve.

Fortunately, we don't need to train lots of different models for different numbers of steps to identify the optimal number.  We just train it once, monitor the validation score, and keep whichever parameters maximize it.  This is called "early stopping".  DeepChem's `ValidationCallback` class can do this for us automatically.  In the example below, we have it compute the validation set's ROC AUC every 1000 training steps.  If you add the `save_dir` argument, it will also save a copy of the best model parameters to disk.

In [8]:
# model = dc.models.MultitaskClassifier(n_tasks=len(tasks),
#                                       n_features=1024,
#                                       layer_sizes=[1000],
#                                       dropouts=0.2,
#                                       learning_rate=0.0001)
# callback = dc.models.ValidationCallback(valid_dataset, 1000, metric)
# model.fit(train_dataset, nb_epoch=50, callbacks=callback)

import deepchem as dc
from deepchem.metrics import Metric, roc_auc_score

# Define model
model = dc.models.MultitaskClassifier(
    n_tasks=len(tasks),
    n_features=1024,
    layer_sizes=[1000],
    dropouts=0.2,
    learning_rate=0.0001
)



class SafeValidationCallback(dc.models.ValidationCallback):
    def __call__(self, model, step, **kwargs):  # Accept arbitrary kwargs safely
        return super().__call__(model, step)

# Define metric
metric = Metric(roc_auc_score, mode="classification")

safe_callback = SafeValidationCallback(
    valid_dataset,
    interval=1000,
    metrics=[metric],
    save_dir= "./best_model"
)

# Train the model
model.fit(train_dataset, nb_epoch=50, callbacks=[safe_callback])

Step 1000 validation: roc_auc_score=0.738485
Step 2000 validation: roc_auc_score=0.751688
Step 3000 validation: roc_auc_score=0.765397
Step 4000 validation: roc_auc_score=0.768383
Step 5000 validation: roc_auc_score=0.768193
Step 6000 validation: roc_auc_score=0.769522
Step 7000 validation: roc_auc_score=0.768754
Step 8000 validation: roc_auc_score=0.771063
Step 9000 validation: roc_auc_score=0.756362
Step 10000 validation: roc_auc_score=0.764463
Step 11000 validation: roc_auc_score=0.762683
Step 12000 validation: roc_auc_score=0.756984
Step 13000 validation: roc_auc_score=0.758867
Step 14000 validation: roc_auc_score=0.764717
Step 15000 validation: roc_auc_score=0.766545
Step 16000 validation: roc_auc_score=0.764013


0.7241657257080079

## Learning Rate Schedules

In the examples above we use a fixed learning rate throughout training.  In some cases it works better to vary the learning rate during training.  To do this in DeepChem, we simply specify a `LearningRateSchedule` object instead of a number for the `learning_rate` argument.  In the following example we use a learning rate that decreases exponentially.  It starts at 0.0002, then gets multiplied by 0.9 after every 1000 steps.

In [9]:
learning_rate = dc.models.optimizers.ExponentialDecay(0.0002, 0.9, 1000)
model = dc.models.MultitaskClassifier(n_tasks=len(tasks),
                                      n_features=1024,
                                      layer_sizes=[1000],
                                      dropouts=0.2,
                                      learning_rate=learning_rate)
model.fit(train_dataset, nb_epoch=50, callbacks=[safe_callback])

Step 1000 validation: roc_auc_score=0.750843
Step 2000 validation: roc_auc_score=0.769939
Step 3000 validation: roc_auc_score=0.767556
Step 4000 validation: roc_auc_score=0.76656
Step 5000 validation: roc_auc_score=0.757987
Step 6000 validation: roc_auc_score=0.766423
Step 7000 validation: roc_auc_score=0.758641
Step 8000 validation: roc_auc_score=0.760702
Step 9000 validation: roc_auc_score=0.764303
Step 10000 validation: roc_auc_score=0.763228
Step 11000 validation: roc_auc_score=0.766989
Step 12000 validation: roc_auc_score=0.765985
Step 13000 validation: roc_auc_score=0.766665
Step 14000 validation: roc_auc_score=0.767194
Step 15000 validation: roc_auc_score=0.766438
Step 16000 validation: roc_auc_score=0.768319


0.541898078918457

# Congratulations! Time to join the Community!

Congratulations on completing this tutorial notebook! If you enjoyed working through the tutorial, and want to continue working with DeepChem, we encourage you to finish the rest of the tutorials in this series. You can also help the DeepChem community in the following ways:

## Star DeepChem on [GitHub](https://github.com/deepchem/deepchem)
This helps build awareness of the DeepChem project and the tools for open source drug discovery that we're trying to build.

## Join the DeepChem Discord
The DeepChem [Discord](https://discord.gg/cGzwCdrUqS) hosts a number of scientists, developers, and enthusiasts interested in deep learning for the life sciences. Join the conversation!

## Citing This Tutorial
If you found this tutorial useful please consider citing it using the provided BibTeX.

In [ ]:
@manual{Intro9,
 title={Advanced Model Training},
 organization={DeepChem},
 author={Eastman, Peter and Ramsundar, Bharath},
 howpublished = {\url{https://github.com/deepchem/deepchem/blob/master/examples/tutorials/Advanced_Model_Training.ipynb}},
 year={2021},
}